In [ ]:
import pandas as pd
import requests
import re
import time

In [14]:
# List of countries and their codes
countries = {
    'Nigeria': 'NGA',
    'South-Africa': 'ZAF',
    'Kenya': 'KEN',
    'Ghana': 'GHA',
    'Ethiopia': 'ETH',
    'Egypt': 'EGY',
    'Morocco': 'MAR',
    'Algeria': 'DZA',
    'Tunisia': 'TUN',
    'Libya': 'LBY',
    'Angola': 'AGO',
    'Zambia': 'ZMB',
    'Zimbabwe': 'ZWE',
    'Mozambique': 'MOZ',
    'Tanzania': 'TZA',
    'Uganda': 'UGA',
    'Rwanda': 'RWA',
    'Senegal': 'SEN',
    'Cameroon': 'CMR',
    'Gabon': 'GAB',
    'Botswana': 'BWA',
    'Namibia': 'NAM',
    'Mali': 'MLI',
    'Burkina-Faso': 'BFA',
    'Niger': 'NER',
    'Chad': 'TCD',
    'Benin': 'BEN',
    'Togo': 'TGO',
    'Sierra-Leone': 'SLE',
    'Liberia': 'LBR',
    'Cabo-Verde': 'CPV'
}

# Empty list to store results
results = []

print("Starting to scrape...\n")

Starting to scrape...



In [15]:
# Loop through each country
for country_name, country_code in countries.items():

    # Build the URL
    url = f"https://www.globalpetrolprices.com/{country_name}/electricity_prices/"

    print(f"Trying {country_name.replace('-', ' '):<35s} ", end="")

    try:
        # Get the web page
        response = requests.get(url, timeout=10)

        # Get the text from the page
        text = response.text

        # Look for pattern: "USD" followed by a number
        pattern = r'USD\s+(\d+\.\d+)'
        found = re.search(pattern, text)

        # If we found it
        if found:
            price = found.group(1)
            price_number = float(price)

            # Only save if price is greater than 0
            if price_number > 0:
                results.append({
                    'country': country_name.replace('-', ' '),
                    'iso': country_code,
                    'tariff_usd_per_kwh': price_number,
                    'source_url': url
                })
                print(f"SUCCESS: ${price_number:.3f}/kWh")
            else:
                print(f"FAILED: Found $0 (invalid data)")
        else:
            print(f"FAILED: No price pattern found")

        # Wait 1 second before next request
        time.sleep(1)

    except:
        print(f"ERROR: Could not access page")
        time.sleep(1)

# Convert to DataFrame
df = pd.DataFrame(results)

print()
print("="*70)
print(f"SCRAPING COMPLETE")
print("="*70)
print(f"Successfully scraped: {len(df)} countries")
print(f"Failed: {len(countries) - len(df)} countries")
print()

# Show results sorted by price
if len(df) > 0:
    df_sorted = df.sort_values('tariff_usd_per_kwh')

    print("SCRAPED DATA (sorted by price):")
    print("-"*70)
    print(f"{'Country':<30s} {'ISO':<5s} {'Tariff ($/kWh)':<15s}")
    print("-"*70)

    for index, row in df_sorted.iterrows():
        print(f"{row['country']:<30s} {row['iso']:<5s} ${row['tariff_usd_per_kwh']:.3f}")

    # Save to CSV
    df_sorted.to_csv('chart3_tariffs_scraped.csv', index=False)
    print()
    print("SAVED: tariffs_scraped.csv")
else:
    print("ERROR: No data was scraped!")

print("="*70)

Trying Nigeria                             SUCCESS: $0.035/kWh
Trying South Africa                        SUCCESS: $0.218/kWh
Trying Kenya                               SUCCESS: $0.222/kWh
Trying Ghana                               SUCCESS: $0.159/kWh
Trying Ethiopia                            SUCCESS: $0.005/kWh
Trying Egypt                               SUCCESS: $0.023/kWh
Trying Morocco                             SUCCESS: $0.127/kWh
Trying Algeria                             SUCCESS: $0.043/kWh
Trying Tunisia                             SUCCESS: $0.068/kWh
Trying Libya                               FAILED: Found $0 (invalid data)
Trying Angola                              SUCCESS: $0.013/kWh
Trying Zambia                              SUCCESS: $0.015/kWh
Trying Zimbabwe                            FAILED: Found $0 (invalid data)
Trying Mozambique                          SUCCESS: $0.127/kWh
Trying Tanzania                            SUCCESS: $0.094/kWh
Trying Uganda                  